In [1]:
from __future__ import print_function
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

import sys
import math
import json
import pickle
import numpy as np

In [2]:
import json
sentence_list_fname = './assignment_training_data_word_segment.json'
sentence_list = json.load(open(sentence_list_fname , 'r',encoding='utf-8'))

In [3]:
sentence_array = []
unicode_word = 0
sentence = sentence_list[4]
for index in range(20):
    print(sentence_list[index])

sentence_times = sentence['times']
sentence_values = sentence['values']
sentence_attributes = sentence['attributes']
result_rule = []
result_rule_index = []
count_index = 0
for word in sentence['indexes']:
    if word == 0:
        result_rule.append('T')
        result_rule_index.append(count_index)
    if word == 2:
        result_rule.append('V')
        result_rule_index.append(count_index)
    if word == 1:
        result_rule.append('A')
        result_rule_index.append(count_index)
    count_index += 1
print(result_rule)
print(result_rule_index)


{'sentenceId': 'c7742348b6c5339cfe970ed3e574e25571dd23e1-eb85e9fb6ec57b2dd9ba53a8cc4b1625b1b90418', 'sentence': '2013年度、2014年度、2015年度，发行人经营性现金流量净额分别为-412,494.25万元、-411,636.30万元以及-530,056.90万元，持续为负且规模较大，主要原因为发行人在开展融资租赁业务时，购买租赁标的一般采用一次性付款，而租金则采用分期收取，现金回收较支出较慢。', 'results': [[0, 8, 11], [4, 8, 15], [2, 8, 13]], 'times': [0, 2, 4], 'values': [11, 13, 15], 'words': ['2013年度', '、', '2014年度', '、', '2015年度', '，', '发行', '人', '经营性现金流量净额', '分别', '为', '-412,494.25万元', '、', '-411,636.30万元', '以及', '-530,056.90万元', '，', '持续', '为', '负且', '规模', '较', '大', '，', '主要', '原因', '为', '发行人', '在', '开展', '融资', '租赁', '业务', '时', '，', '购买', '租赁', '标的', '一般', '采用', '一次性', '付款', '，', '而', '租金', '则', '采用', '分期', '收取', '，', '现金', '回收', '较', '支出', '较', '慢', '。'], 'attributes': [8], 'indexes': [0, 6, 0, 6, 0, 7, 17, 18, 1, 11, 8, 2, 6, 2, 92, 2, 7, 134, 8, 2804, 52, 19, 47, 7, 23, 54, 8, 28, 45, 600, 104, 146, 33, 868, 7, 317, 146, 918, 1078, 437, 2326, 712, 7, 186, 542, 1191, 437, 1929, 759, 7, 132, 688, 19, 257, 19, 976

In [4]:
special_array = [88,14,19,97,21,360,22,40,1796,309,102]
cut_char_array = [7,10,26]
change_word = [22,32,12,261,546,49]
change_s_word = [141,644]

def check_result(sen_index):
    sentence = sentence_list[sen_index]
    results = []
    for time in sentence['times']:
        for attribute in sentence['attributes']:
            for value in sentence['values']:
                results.append([time, attribute, value])
    indexes = sentence['indexes']
    for index in range(len(indexes) - 3):
        if (indexes[index] == 2) and (indexes[index + 1] == 7) and (indexes[index + 2] == 2) :
            indexes[index + 1] = 6
        if (indexes[index] == 7) and (indexes[index + 1] == 11) and (indexes[index + 2] == 8) :
            indexes[index] = 6
            
    final_result = []
    not_in_ssen = 0
    for result in results:
        #过滤V
        if indexes[result[2] - 1] in change_word:
            continue
        if indexes[result[2] + 1] in change_s_word:
            continue
        
        
        #顺序TAV
        #if result[0] >=  result[1]:
         #   continue
        if result[1] >=  result[2]:
            continue
             
        #过滤AV不在同一个短语的
        counter = result[1]
        while counter <= result[2]:
            if (indexes[counter] in cut_char_array):
                #if (indexes[counter] == 7 and result[2] <= counter + 3):
                 #   continue
                not_in_ssen = 1
                break
            counter += 1
            
        if not_in_ssen == 1:
            not_in_ssen = 0
            continue
        
        #过滤短语内有特殊内容的
        counterb = result[1]
        while 1:
            counterb -= 1
            if indexes[counterb] in cut_char_array:       
                break
            if counterb == 0 or counterb == -1:
                break
        countere = result[2]
        while 1:
            countere += 1
            if (indexes[countere] in cut_char_array):
               # if (indexes[countere] == 7 and result[2] <= countere + 3):
                #    continue
                break
                
        special_char = 0
        counter = counterb
        while counter <= countere:
            if indexes[counter] in special_array:  
                special_char = 1
                break
            counter += 1        
        if special_char == 1:
            continue   
            
        #处理T和AV的对应关系
        num_v = 0
        res_v_index = 0    
        counter = counterb
        while counter <= countere:
            if indexes[counter] == 2:       
                num_v += 1
            if counter == result[2]: 
                res_v_index = num_v
            counter += 1
        search_t_index = num_v - res_v_index
        counter = result[1]
        result_t = 0
        while counter >= 0:
            if indexes[counter] == 0:  
                if search_t_index <= 0:
                    result_t = counter
                    break
                else :
                    search_t_index -= 1
            counter -= 1  
        if result[0] != result_t:
            continue
        
        final_result.append(result)
    counter = 0
    while counter < (len(final_result) - 1):
        if final_result[counter][2] == final_result[counter + 1][2]:  
            del final_result[counter]
        else:
            counter += 1
    return final_result  

final_result = []
for i in range(len(sentence_list)): 
    final_result.append(check_result(i))
print(final_result)

[[[0, 8, 11], [2, 8, 13], [4, 8, 15]], [], [[0, 10, 13], [2, 10, 15], [4, 10, 17]], [[0, 8, 11], [2, 8, 13], [4, 8, 15]], [[1, 11, 13], [1, 15, 17], [19, 22, 23], [19, 25, 26]], [], [[0, 5, 6], [0, 8, 9]], [[0, 9, 12], [2, 9, 14], [4, 9, 16], [6, 9, 18]], [[0, 7, 9], [2, 7, 11], [4, 7, 13]], [[0, 7, 10], [0, 21, 24], [2, 7, 12], [2, 21, 26], [4, 7, 14], [4, 21, 28]], [[6, 15, 17], [8, 15, 19], [10, 15, 21]], [[0, 9, 12], [2, 9, 14], [4, 9, 16], [6, 9, 18]], [[1, 8, 11], [3, 8, 13], [5, 8, 15]], [[0, 5, 9], [2, 5, 11], [4, 5, 13]], [[27, 38, 41], [29, 38, 43], [31, 38, 45], [33, 38, 47]], [], [], [[1, 9, 12], [3, 9, 14], [5, 9, 16]], [[0, 18, 21], [2, 18, 23], [4, 18, 25]], [[1, 7, 10]], [[11, 19, 22], [13, 19, 24], [15, 19, 26], [18, 19, 28]], [[1, 9, 12], [3, 9, 14], [5, 9, 16]], [[0, 7, 10], [2, 7, 12], [4, 7, 14]], [[1, 9, 12], [3, 9, 14], [5, 9, 16]], [[1, 9, 13], [3, 9, 15], [5, 9, 17]], [[1, 9, 13], [3, 9, 15], [5, 9, 17]], [[1, 9, 14], [3, 9, 16], [5, 9, 18]], [], [], [], [[1, 8

In [5]:
for i in range(len(sentence_list)): 
    sentence_list[i]['pred_results'] = final_result[i]
file_out_train = open("output_train.json", "w")
file_out_train.write(json.dumps(sentence_list))
file_out_train.close()
print(sentence_list[0])

{'sentenceId': 'c7742348b6c5339cfe970ed3e574e25571dd23e1-eb85e9fb6ec57b2dd9ba53a8cc4b1625b1b90418', 'sentence': '2013年度、2014年度、2015年度，发行人经营性现金流量净额分别为-412,494.25万元、-411,636.30万元以及-530,056.90万元，持续为负且规模较大，主要原因为发行人在开展融资租赁业务时，购买租赁标的一般采用一次性付款，而租金则采用分期收取，现金回收较支出较慢。', 'results': [[0, 8, 11], [4, 8, 15], [2, 8, 13]], 'times': [0, 2, 4], 'values': [11, 13, 15], 'words': ['2013年度', '、', '2014年度', '、', '2015年度', '，', '发行', '人', '经营性现金流量净额', '分别', '为', '-412,494.25万元', '、', '-411,636.30万元', '以及', '-530,056.90万元', '，', '持续', '为', '负且', '规模', '较', '大', '，', '主要', '原因', '为', '发行人', '在', '开展', '融资', '租赁', '业务', '时', '，', '购买', '租赁', '标的', '一般', '采用', '一次性', '付款', '，', '而', '租金', '则', '采用', '分期', '收取', '，', '现金', '回收', '较', '支出', '较', '慢', '。'], 'attributes': [8], 'indexes': [0, 6, 0, 6, 0, 7, 17, 18, 1, 11, 8, 2, 6, 2, 92, 2, 7, 134, 8, 2804, 52, 19, 47, 7, 23, 54, 8, 28, 45, 600, 104, 146, 33, 868, 7, 317, 146, 918, 1078, 437, 2326, 712, 7, 186, 542, 1191, 437, 1929, 759, 7, 132, 688, 19, 257, 19, 976

In [6]:
info = json.load(open("./assignment_training_data_word_segment.json", 'r'))  # original data json
answers = json.load(open("output_train.json", 'r')) 

n_info = len(info)
n_answers = len(answers)

prec, recall = 0., 0.
post, negt, posf = 0, 0, 0

n = n_answers

if n_info == n_answers:
    print ("No missing or flowing problem")
else:
    if n_info < n_answers:
        n = n_info
        print ("flowing problems")
    else :
        n = n_answers
        print ("missing problems")

for i in range(0, n):
    for r in info[i]['results']:
        if (r in answers[i]['pred_results']):
            post = post + 1
        else :
            posf = posf + 1
    for r in answers[i]['pred_results']:
        if (r not in info[i]['results']):
            negt = negt + 1

recall = post * 1. / (post + posf)
prec   = post * 1. / (post + negt)
correct = post * 1. / (post + posf + negt)
f1 = (prec * recall) * 2. / (prec + recall)

print("recall:%.2lf prec:%.2lf correct:%.2lf F1:%.2lf" % (recall, prec, correct, f1))

No missing or flowing problem
recall:0.99 prec:0.99 correct:0.98 F1:0.99


In [7]:
test_sentence_list_fname = './assignment_test_data_word_segment.json'
test_sentence_list = json.load(open(test_sentence_list_fname , 'r',encoding='utf-8'))
test_final_result = []
for i in range(len(test_sentence_list)): 
    test_final_result.append(check_result(i))
print(test_final_result)

[[[0, 8, 11], [2, 8, 13], [4, 8, 15]], [], [[0, 10, 13], [2, 10, 15], [4, 10, 17]], [[0, 8, 11], [2, 8, 13], [4, 8, 15]], [[1, 11, 13], [1, 15, 17], [19, 22, 23], [19, 25, 26]], [], [[0, 5, 6], [0, 8, 9]], [[0, 9, 12], [2, 9, 14], [4, 9, 16], [6, 9, 18]], [[0, 7, 9], [2, 7, 11], [4, 7, 13]], [[0, 7, 10], [0, 21, 24], [2, 7, 12], [2, 21, 26], [4, 7, 14], [4, 21, 28]], [[6, 15, 17], [8, 15, 19], [10, 15, 21]], [[0, 9, 12], [2, 9, 14], [4, 9, 16], [6, 9, 18]], [[1, 8, 11], [3, 8, 13], [5, 8, 15]], [[0, 5, 9], [2, 5, 11], [4, 5, 13]], [[27, 38, 41], [29, 38, 43], [31, 38, 45], [33, 38, 47]], [], [], [[1, 9, 12], [3, 9, 14], [5, 9, 16]], [[0, 18, 21], [2, 18, 23], [4, 18, 25]], [[1, 7, 10]], [[11, 19, 22], [13, 19, 24], [15, 19, 26], [18, 19, 28]], [[1, 9, 12], [3, 9, 14], [5, 9, 16]], [[0, 7, 10], [2, 7, 12], [4, 7, 14]], [[1, 9, 12], [3, 9, 14], [5, 9, 16]], [[1, 9, 13], [3, 9, 15], [5, 9, 17]], [[1, 9, 13], [3, 9, 15], [5, 9, 17]], [[1, 9, 14], [3, 9, 16], [5, 9, 18]], [], [], [], [[1, 8

In [8]:
for i in range(len(test_sentence_list)): 
    test_sentence_list[i]['results'] = test_final_result[i]
file_out_test = open("output_test.json", "w")
file_out_test.write(json.dumps(test_sentence_list))
for i in range(20): 
    print(test_sentence_list[i])
file_out_test.close()

{'sentenceId': '76d78d5af70bb6608fd5ef731922e6036eda6d52-20248dad98e94d3180de3d525e74fc7ba0cea465', 'sentence': '截至2015年12月31日，碧桂园控股累计对外担保金额约为6,241,806万元，剔除为物业买方向银行申请按揭贷款而提供的短期过渡性担保，碧桂园控股累计对外担保金额约为178,190万元，仅占期末归属于母公司所有者权益的2.73%。', 'times': [1], 'values': [10, 35, 45], 'words': ['截至', '2015年12月31日', '，', '碧桂园', '控股', '累计', '对外担保', '金额', '约', '为', '6,241,806万元', '，', '剔除', '为', '物业', '买方', '向', '银行', '申请', '按揭', '贷款', '而', '提供', '的', '短期', '过渡性', '担保', '，', '碧桂园', '控股', '累计', '对外担保', '金额', '约', '为', '178,190万元', '，', '仅', '占', '期末', '归属', '于', '母公司', '所有者权益', '的', '2.73%', '。'], 'attributes': [6, 31, 43], 'indexes': [15, 0, 7, 677, 256, 315, 1, 51, 357, 8, 2, 7, 1108, 8, 223, 2316, 462, 149, 2557, 1236, 209, 186, 248, 12, 105, 2691, 203, 7, 677, 256, 315, 1, 51, 357, 8, 2, 7, 764, 14, 41, 302, 112, 94, 1, 12, 2, 10], 'results': [[0, 8, 11], [2, 8, 13], [4, 8, 15]]}
{'sentenceId': 'e71826135a20306ebc52acba1c5c793716c8d81a-8664c9cd649bfb47ce153b98d0ff6b1845e71629', 'sentence': '2012年度、201